In [65]:
import pandas as pd
import numpy as np
import math

In [66]:
# Load CSV
file_path = "output/step-2-weekly-highs-and-lows.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df.tail()

,Date,OPEN,HIGH,LOW,PREV. CLOSE,close,Day,Week,Order Type,Buy Quantity,Investment,Total Share,Total Investment,Average Price,Target,IRLower,IRUpper,Description
297,2024-12-23,"1,782.00","1,806.00","1,781.30","1,771.50","1,801.00",Monday,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,2024-12-24,"1,802.05","1,808.70","1,789.55","1,801.00","1,798.10",Tuesday,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,2024-12-26,"1,798.55","1,812.00","1,780.75","1,798.10","1,790.75",Thursday,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,2024-12-27,"1,794.80","1,805.95","1,789.10","1,790.75","1,798.25",Friday,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,NaN,NaN,"1,812.00","1,780.75",NaN,NaN,WEEKLY SUMMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# Columns to clean
cols_to_convert = ["OPEN", "HIGH", "LOW", "PREV. CLOSE", "close"]

# Convert columns: Remove commas and change to float
for col in cols_to_convert:
    df[col] = df[col].astype(str).str.replace(',', '', regex=True).astype(float)

df.head()

,Date,OPEN,HIGH,LOW,PREV. CLOSE,close,Day,Week,Order Type,Buy Quantity,Investment,Total Share,Total Investment,Average Price,Target,IRLower,IRUpper,Description
0,2024-12-30,1792.20,1815.00,1771.0,1798.25,1777.90,Monday,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-12-31,1770.30,1781.80,1765.6,1777.90,1772.85,Tuesday,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-01,1773.45,1794.00,1761.1,1772.85,1782.75,Wednesday,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-02,1777.10,1803.45,1773.0,1782.75,1793.75,Thursday,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-03,1791.00,1795.25,1746.3,1793.75,1749.20,Friday,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
INVESTMENT_PER_TRADE = 5000
TARGET_MULTIPLIER = 1.06
UPPER_MULTIPLIER = 1.0275
LOWER_MULTIPLIER = 0.9725

In [69]:
# Initialize tracking variables
total_shares = 0
total_investment = 0
avg_price = 0
prev_week_high = None
active_buys = []  # Track open positions

updated_rows = []
weekIdx = 0
# Process week by week
for week, group in df.groupby('Week'):
    
    
    weekIdx += 1
    weekly_high = group['HIGH'].max()
    weekly_low = group['LOW'].min()

    buy_triggered = False
    buy_reason = ""

    for _, row in group.iterrows():
        if weekIdx == 1:
            weekIdx += 1
            prev_week_high = group['HIGH'].max()
            updated_rows.append({
                **row,
            })
            continue  # Skip first week

        
        # Check if we should sell first
        if total_shares > 0 and row['HIGH'] >= avg_price * TARGET_MULTIPLIER:
            sell_price = avg_price * TARGET_MULTIPLIER
            profit = (sell_price - avg_price) * total_shares
            
            updated_rows.append({
                **row,
                "Order Type": "SELL",
                "Buy Quantity": "",
                "Investment": "",
                "Total Share": 0,
                "Total Investment": 0,
                "Average Price": "",
                "Target": "",
                "IRLower": "",
                "IRUpper": "",
                "Description": f"Sold {total_shares} shares at {sell_price}, Profit: {profit}"
            })
            
            total_shares = 0
            total_investment = 0
            avg_price = 0
            active_buys = []
            continue
        
        # Determine buy price
        if total_shares == 0:
            buy_price = prev_week_high
            buy_reason = "Prev week high - Buy price"
        else:
            if row['LOW'] <= LOWER_MULTIPLIER * avg_price:
                buy_price = LOWER_MULTIPLIER * avg_price
                buy_reason = "Buy Triggered at IRLower"
            elif row['HIGH'] >= UPPER_MULTIPLIER * avg_price:
                buy_price = UPPER_MULTIPLIER * avg_price
                buy_reason = "Buy Triggered at IRUpper"
            else:
                buy_reason = "No buy triggered"
                updated_rows.append({
                    **row,
                })
                continue  # No buy condition met
        
        buy_quantity = math.floor(INVESTMENT_PER_TRADE / buy_price)
        investment = buy_quantity * buy_price

        total_shares += buy_quantity
        total_investment += investment
        avg_price = total_investment / total_shares

        target_price = avg_price * TARGET_MULTIPLIER
        upper_range = avg_price * UPPER_MULTIPLIER
        lower_range = avg_price * LOWER_MULTIPLIER

        active_buys.append({
            "Order Type": "BUY",
            "Week": week,
            "Buy Price": buy_price,
            "Buy Quantity": buy_quantity,
            "Investment": investment,
            "Total Shares": total_shares,
            "Total Investment": total_investment,
            "Avg Price": avg_price,
            "Target": target_price,
            "IRLower": lower_range,
            "IRUpper": upper_range,
            "Description": buy_reason
        })

        updated_rows.append({
            **row,
            "Buy Quantity": buy_quantity,
            "Investment": investment,
            "Total Share": total_shares,
            "Total Investment": total_investment,
            "Average Price": avg_price,
            "Target": target_price,
            "IRLower": lower_range,
            "IRUpper": upper_range,
            "Description": buy_reason
        })
    
    updated_rows.append({
        "Date": "",
        "Day": "WEEKLY SUMMARY",
        "HIGH": weekly_high,
        "LOW": weekly_low,
        "Buy Quantity": "",
        "Investment": "",
        "Total Share": "",
        "Total Investment": "",
        "Average Price": "",
        "Target": "",
        "IRLower": "",
        "IRUpper": "",
        "Description": f"Week {week} Summary: High={weekly_high}, Low={weekly_low}"
    })

    prev_week_high = weekly_high

# Create new DataFrame with updated data
df_updated = pd.DataFrame(updated_rows)
df_updated.head(30)
df = df_updated

In [76]:
# Initialize tracking variables
total_shares = 0
total_investment = 0
avg_price = 0
prev_week_high = None
active_buys = []  # Track open positions

updated_rows = []
weekIdx = 0
# Process week by week
for week, group in df.groupby('Week'):
    
    
    weekIdx += 1
    weekly_high = group['HIGH'].max()
    weekly_low = group['LOW'].min()

    buy_triggered = False
    buy_reason = ""

    for _, row in group.iterrows():
        if weekIdx == 1:
            weekIdx += 1
            prev_week_high = group['HIGH'].max()
            updated_rows.append({
                **row,
            })
            continue  # Skip first week

        
        # Check if we should sell first
        if total_shares > 0 and row['HIGH'] >= avg_price * TARGET_MULTIPLIER:
            sell_price = avg_price * TARGET_MULTIPLIER
            profit = (sell_price - avg_price) * total_shares
            
            updated_rows.append({
                **row,
                "Order Type": "SELL",
                "Buy Quantity": "",
                "Investment": "",
                "Total Share": 0,
                "Total Investment": 0,
                "Average Price": "",
                "Target": "",
                "IRLower": "",
                "IRUpper": "",
                "Description": f"Sold {total_shares} shares at {sell_price}, Profit: {profit}",
                "Profit": profit,
                "Capital Employed": total_shares * sell_price,
                "Profit Percentage": (profit / (total_shares * avg_price)) * 100 
            })
            
            total_shares = 0
            total_investment = 0
            avg_price = 0
            active_buys = []
            continue
        
        # Determine buy price
        if total_shares == 0:
            buy_price = prev_week_high
            buy_reason = "Prev week high - Buy price"
        else:
            if row['LOW'] <= LOWER_MULTIPLIER * avg_price:
                buy_price = LOWER_MULTIPLIER * avg_price
                buy_reason = "Buy Triggered at IRLower"
            elif row['HIGH'] >= UPPER_MULTIPLIER * avg_price:
                buy_price = UPPER_MULTIPLIER * avg_price
                buy_reason = "Buy Triggered at IRUpper"
            else:
                buy_reason = "No buy triggered"
                updated_rows.append({
                    **row,
                })
                continue  # No buy condition met
        
        buy_quantity = math.floor(INVESTMENT_PER_TRADE / buy_price)
        investment = buy_quantity * buy_price

        total_shares += buy_quantity
        total_investment += investment
        avg_price = total_investment / total_shares

        target_price = avg_price * TARGET_MULTIPLIER
        upper_range = avg_price * UPPER_MULTIPLIER
        lower_range = avg_price * LOWER_MULTIPLIER

        active_buys.append({
            "Order Type": "BUY",
            "Week": week,
            "Buy Price": buy_price,
            "Buy Quantity": buy_quantity,
            "Investment": investment,
            "Total Shares": total_shares,
            "Total Investment": total_investment,
            "Avg Price": avg_price,
            "Target": target_price,
            "IRLower": lower_range,
            "IRUpper": upper_range,
            "Description": buy_reason
        })

        updated_rows.append({
            **row,
            "Order Type": "BUY",
            "Buy Quantity": buy_quantity,
            "Investment": investment,
            "Total Share": total_shares,
            "Total Investment": total_investment,
            "Average Price": avg_price,
            "Target": target_price,
            "IRLower": lower_range,
            "IRUpper": upper_range,
            "Description": buy_reason
        })
    
    updated_rows.append({
        "Date": "",
        "Day": "WEEKLY SUMMARY",
        "HIGH": weekly_high,
        "LOW": weekly_low,
        "Buy Quantity": "",
        "Investment": "",
        "Total Share": "",
        "Total Investment": "",
        "Average Price": "",
        "Target": "",
        "IRLower": "",
        "IRUpper": "",
        "Description": f"Week {week} Summary: High={weekly_high}, Low={weekly_low}"
    })

    prev_week_high = weekly_high

# Create new DataFrame with updated data
df_updated = pd.DataFrame(updated_rows)
df_updated.head(30)
df = df_updated

In [77]:
columns_to_round = ['Buy Quantity', 'Investment', 'Total Share', 'Total Investment',
                    'Average Price', 'Target', 'IRLower', 'IRUpper']

df[columns_to_round] = df[columns_to_round].apply(
       lambda x: pd.to_numeric(x, errors='coerce').round(3)
)

df.to_csv("output/step-3-buy-orders.csv")